# Analysis of the evaluation result of weather sonification

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from model.WeatherMusicAssociation import WeatherMusicAssociationModel
from music21 import converter
from tqdm.notebook import tqdm
from common import common


In [ ]:
common.init_notebook()

In [ ]:
out_dir = common.data_path("output")
midi_dir = common.data_path("midi_for_evaluation_2")

In [ ]:
# The palette name to be used by the seaborn charts.
palette="muted"

In [ ]:
key_encoding = "onehot"

model = WeatherMusicAssociationModel(key_encoding)
model.load_trained_models()
df = model.load_weather_music_features()

In [ ]:
df

In [ ]:
# Check the mode of generated weather music.
weather_10_df = None

mydf = pd.read_csv(f"{midi_dir}/samples.csv")
if weather_10_df is None:
  weather_10_df = mydf
else:
  weather_10_df = pd.concat([weather_10_df, mydf])


In [ ]:
# joined_df = weather_10_df.join(df.set_index('date_time'), on='date_time', how='left')
joined_df = weather_10_df

In [ ]:
joined_df

In [ ]:
files = sorted(os.listdir(midi_dir))
midi_data_list = [converter.parse(os.path.join(midi_dir,file)) for file in tqdm(files) if file.endswith(".mid")]

analyzed_keys = [s.analyze('key') for s in tqdm(midi_data_list)]

In [ ]:
joined_df.shape

In [ ]:
joined_df['reanalysed_mode'] = [k.mode for k in analyzed_keys]
joined_df['reanalysed_mode_correlationCoefficient'] = [k.correlationCoefficient for k in analyzed_keys]


In [ ]:
joined_df.to_csv(f"{out_dir}/joined_df_of_50_samples.csv")

In [ ]:
joined_df[['mode_str', 'reanalysed_mode', 'reanalysed_mode_correlationCoefficient']]

In [ ]:
joined_df[joined_df['mode_str'] == joined_df['reanalysed_mode']]

In [ ]:
joined_df[joined_df['mode_str'] != joined_df['reanalysed_mode']]

In [ ]:
# Visualise the samples for evaluation on the latent emotion space.
sns.scatterplot(joined_df, x='Valence', y='Arousal', hue="pattern", palette=palette, s=30)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
joined_df[['pattern', 'Valence', 'Arousal']].groupby("pattern", as_index=False).describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 6))
sns.boxplot(x='pattern', y='Valence', data=joined_df, ax=axes[0], color="yellow")
axes[0].tick_params(axis='x', rotation=45)
sns.boxplot(x='pattern', y='Arousal', data=joined_df, ax=axes[1], color="yellow")
axes[1].tick_params(axis='x', rotation=45)
plt.show()


In [ ]:
joined_df.columns.to_list()

In [ ]:
x_vars = ['mean_note_duration', 'tempo', 'pitch_range']
cols = x_vars + ['pattern']
sns.pairplot(joined_df[cols], hue='pattern', x_vars=x_vars, diag_kind='hist')